In [1]:
import sys
import os
from pathlib import Path
import numpy as np
import pandas as pd
import skimage.io as io
import torch
import time
from torch.utils.data import DataLoader
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.ops.boxes import nms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pytorch_toolbelt.utils import to_numpy, rle_encode
from dotenv import load_dotenv
from easydict import EasyDict
from tqdm import tqdm

from src.visualization import plot_two_masks, plot_mask_bbox
from src.postprocessing import remove_overlapping_pixels, postprocess_predictions
from src.dataset import CellDataset
from src.augmentations import train_transform
from src.utils import images2device, targets2device
from src.iou_metric import fast_iou

current_dir = Path(".")
load_dotenv()

True

In [2]:
weights_name = 'maskrcnn-first_baseline_with_metric-2021-12-14 08:32:09.520121.ckpt'
weights_path = Path("weights") / weights_name

In [3]:
device = 'cuda:1'
config = EasyDict(
    dataset_path=Path(os.environ["dataset_path"]),
    device="cuda:1",
    val_size=0.2,
    batch_size=6,
    num_workers=30,
    epochs=1,
    mask_threshold=0.5,
    score_threshold=0.2,
    nms_threshold=None,
)

In [4]:
model = maskrcnn_resnet50_fpn(progress=False, num_classes=2)
model.load_state_dict(torch.load(weights_path, map_location=torch.device(device)))
model.to(device)
model.eval()
print()

In [5]:
train_dataloader = DataLoader(
    dataset=CellDataset(cfg=config, mode="train", transform=train_transform),
    num_workers=30,
    batch_size=config.batch_size,
    shuffle=True,
    collate_fn=lambda x: tuple(zip(*x)),
)

In [9]:
def evaluate_thresholds(trial):
    mask_thr = trial.suggest_float('mask_thr', 0.0, 0.95)
    score_thr = trial.suggest_float('score_thr', 0.0, 0.95)
    nms_thr = trial.suggest_float('nms_thr', 0.0, 1.0)
    ious = []
    for batch_idx, (images, targets) in enumerate(tqdm(train_dataloader)):
        images= images2device(images, device)
        outputs = model(images)
        processed_outputs = postprocess_predictions(outputs, mask_thr, score_thr, nms_thr)
        for i in range(len(images)):
            predicted_masks = processed_outputs[i]['masks']
            true_masks = targets[i]['masks']
            x = time.time()
            print(predicted_masks.shape, true_masks.shape)
            iou = fast_iou(predicted_masks, true_masks.numpy())
            print(time.time() - x)
            ious.append(iou)
        if batch_idx >= 4:
            break
    return np.mean(ious)

In [ ]:
os.makedirs('thresholds', exist_ok=True)

In [16]:
import optuna
study = optuna.create_study()
for i in range(5):
    study.optimize(evaluate_thresholds, n_trials=1)
    with open("thresholds/first_baseline_with_metric.pickle", 'wb') as file:
        pickle.dump(study, file)
    study.trials_dataframe().to_csv("thresholds/first_baseline_with_metric.csv")

,number,value,datetime_start,datetime_complete,duration,params_mask_thr,params_nms_thr,params_score_thr,state
0,0,None,2021-12-14 19:25:59.884968,None,None,0.922141,0.267078,0.695879,RUNNING
